# ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್

ಈ ಕೋಡ್ ಮಾದರಿಯಲ್ಲಿ, ನೀವು [ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ಫ್ರೇಮ್ವರ್ಕ್ ಅನ್ನು ಬಳಸಿಕೊಂಡು ಒಂದು ಮೂಲ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುತ್ತೀರಿ.

ಈ ಮಾದರಿಯ ಉದ್ದೇಶವು ವಿವಿಧ ಏಜೆಂಟಿಕ್ ಮಾದರಿಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುವಾಗ ನಾವು ನಂತರ ಬಳಸುವ ಹಂತಗಳನ್ನು ನಿಮಗೆ ತೋರಿಸುವುದಾಗಿದೆ.


## ಅಗತ್ಯವಿರುವ ಪೈಥಾನ್ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## ಕ್ಲೈಂಟ್ ರಚನೆ

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ಅನ್ನು LLM ಗೆ ಪ್ರವೇಶಕ್ಕಾಗಿ ಬಳಸುತ್ತೇವೆ.

`ai_model_id` ಅನ್ನು `gpt-4o-mini` ಎಂದು ವ್ಯಾಖ್ಯಾನಿಸಲಾಗಿದೆ. GitHub Models ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಲಭ್ಯವಿರುವ ಮತ್ತೊಂದು ಮಾದರಿಗೆ ಮಾದರಿಯನ್ನು ಬದಲಾಯಿಸಲು ಪ್ರಯತ್ನಿಸಿ ಮತ್ತು ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳನ್ನು ನೋಡಿ.

GitHub Models ಗೆ `base_url` ಬಳಸಲು ನಾವು ಬಳಸುವ `Azure Inference SDK` ಅನ್ನು ಬಳಸಲು, ನಾವು Semantic Kernel ನಲ್ಲಿ `OpenAIChatCompletion` ಕನೆಕ್ಟರ್ ಅನ್ನು ಬಳಸುತ್ತೇವೆ. ಇತರ ಮಾದರಿ ಪೂರೈಕೆದಾರರಿಗಾಗಿ Semantic Kernel ಅನ್ನು ಬಳಸಲು [ಲಭ್ಯವಿರುವ ಇತರ ಕನೆಕ್ಟರ್‌ಗಳು](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ಕೂಡ ಇವೆ.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುವುದು

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ ನಾವು `TravelAgent` ಎಂಬ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುತ್ತೇವೆ.

ಈ ಉದಾಹರಣೆಗೆ, ನಾವು ತುಂಬಾ ಸರಳವಾದ ಸೂಚನೆಗಳನ್ನು ಬಳಸುತ್ತಿದ್ದೇವೆ. ನೀವು ಈ ಸೂಚನೆಗಳನ್ನು ಬದಲಾಯಿಸಿ ಏಜೆಂಟ್ ಹೇಗೆ ವಿಭಿನ್ನವಾಗಿ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಬಹುದು.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ಏಜೆಂಟ್ ಅನ್ನು ಚಲಿಸುವುದು

ಈಗ ನಾವು `ChatHistoryAgentThread` ಪ್ರಕಾರದ ಥ್ರೆಡ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವ ಮೂಲಕ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಿಸಬಹುದು. ಅಗತ್ಯವಿರುವ ಸಿಸ್ಟಮ್ ಸಂದೇಶಗಳನ್ನು ಏಜೆಂಟ್‌ನ invoke_stream `messages` ಕೀವರ್ಡ್ ಆರ್ಗ್ಯುಮೆಂಟ್‌ಗೆ ಒದಗಿಸಲಾಗುತ್ತದೆ.

ಇವುಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿದ ನಂತರ, ನಾವು `user_inputs` ಅನ್ನು ರಚಿಸುತ್ತೇವೆ, ಇದು ಬಳಕೆದಾರನು ಏಜೆಂಟ್‌ಗೆ ಕಳುಹಿಸುತ್ತಿರುವುದು. ಈ ಸಂದರ್ಭದಲ್ಲಿ, ನಾವು ಈ ಸಂದೇಶವನ್ನು `Plan me a sunny vacation` ಎಂದು ಹೊಂದಿಸಿದ್ದೇವೆ.

ಈ ಸಂದೇಶವನ್ನು ಬದಲಾಯಿಸಲು ಮುಕ್ತವಾಗಿರಿ, ಏಜೆಂಟ್ ಹೇಗೆ ವಿಭಿನ್ನವಾಗಿ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಲು.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿಕೊಂಡು ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅನ್ವಯಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
